# Multi XScience

In [1]:
import re

from datasets import load_dataset, load_metric
import evaluate
import nltk
import nltk.data
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
from transformers import (
    AdamW, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

In [2]:
torch.backends.mps.is_available()

True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/luka/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
DATASET_NAME = "multi_x_science_sum"
DOC_SEP = " ||||| "
BATCH_SIZE = 64
MAX_LENGTH_ENC = 512  #4096
MAX_LENGTH_DEC = 256
# N = 16

## Set up evaluation

In [5]:
rouge = load_metric("rouge")

/var/folders/fs/_8_tbmdx5036fk6n1h6v9b680000gn/T/ipykernel_24524/4132584981.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")


## Load dataset

In [6]:
dataset = load_dataset(DATASET_NAME)

Found cached dataset multi_x_science_sum (/Users/luka/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729)


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 30369
    })
    test: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 5093
    })
    validation: Dataset({
        features: ['aid', 'mid', 'abstract', 'related_work', 'ref_abstract'],
        num_rows: 5066
    })
})

In [8]:
dataset["train"][0]

{'aid': 'math9912167',
 'mid': '1631980677',
 'abstract': 'Author(s): Kuperberg, Greg; Thurston, Dylan P. | Abstract: We give a purely topological definition of the perturbative quantum invariants of links and 3-manifolds associated with Chern-Simons field theory. Our definition is as close as possible to one given by Kontsevich. We will also establish some basic properties of these invariants, in particular that they are universally finite type with respect to algebraically split surgery and with respect to Torelli surgery. Torelli surgery is a mutual generalization of blink surgery of Garoufalidis and Levine and clasper surgery of Habiro.',
 'related_work': 'Two other generalizations that can be considered are invariants of graphs in 3-manifolds, and invariants associated to other flat connections @cite_16 . We will analyze these in future work. Among other things, there should be a general relation between flat bundles and links in 3-manifolds on the one hand and finite covers and b

## Format dataset to our needs

In [9]:
pat = re.compile("@cite_[0-9]+")

In [10]:
def preprocess_dataset_batched(example):
    output = {}
    output["abstracts"] = []
    output["related_work"] = []
    
    for abstract, ref_abstract in zip(
        example["abstract"], example["ref_abstract"]
    ):
        output["abstracts"].append(
            abstract.split("| Abstract: ")[-1]
            + DOC_SEP
            + DOC_SEP.join([x for x in ref_abstract["abstract"] if x])
        )
    for related_work in example["related_work"]:
        output["related_work"].append(pat.sub("@cite", related_work))
    
    return output

In [11]:
dataset_processed = (
    dataset
    # .filter(lambda _, idx: idx < N, with_indices=True)
    .map(
        # preprocess_dataset,
        preprocess_dataset_batched,
        remove_columns=dataset["train"].column_names,
        batched=True,
        batch_size=BATCH_SIZE,
    )
)

Loading cached processed dataset at /Users/luka/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729/cache-f3232cb45e6040e6.arrow
Loading cached processed dataset at /Users/luka/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729/cache-da1a80df5f4bf131.arrow
Loading cached processed dataset at /Users/luka/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729/cache-238661e6a318eda9.arrow


In [12]:
dataset_processed

DatasetDict({
    train: Dataset({
        features: ['related_work', 'abstracts'],
        num_rows: 30369
    })
    test: Dataset({
        features: ['related_work', 'abstracts'],
        num_rows: 5093
    })
    validation: Dataset({
        features: ['related_work', 'abstracts'],
        num_rows: 5066
    })
})

## Model 2: Fine-tune Centrum

In [13]:
CHECKPOINT = "ratishsp/Centrum"

In [14]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
model = AutoModelForSeq2SeqLM.from_pretrained(CHECKPOINT)

In [15]:
tokenizer.add_tokens(DOC_SEP, special_tokens=True)
model.resize_token_embeddings(len(tokenizer))
docsep_token_id = tokenizer.convert_tokens_to_ids(DOC_SEP)

In [16]:
def tokenize_dataset_batched(example):
    # Tokenizer input
    output = tokenizer(
        example["abstracts"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH_ENC,
        return_tensors="pt",
    )
    
    # Tokenizer output
    output["labels"] = (
        tokenizer(
            example["related_work"],
            padding="max_length",
            truncation=True,
            max_length=MAX_LENGTH_DEC,
            return_tensors="pt",
            return_attention_mask=False,
        )
        .input_ids
    )
    # Tokenizer output ignore padding in loss function
    # torch ignore -100 in loss function computation
    output["labels"] = [
        [
            -100 if token == tokenizer.pad_token_id else token
            for token in labels
        ]
        for labels in output["labels"]
    ]
    
    # Global attention
    output["global_attention_mask"] = np.array(
        [
            [
                1 if token in (tokenizer.cls_token_id, docsep_token_id) else 0 
                for token in each
            ]
            for each in output["input_ids"]
        ], 
        dtype=np.float32,
    )
    
    return output

In [17]:
dataset_tokenized = (
    dataset_processed
    .map(
        tokenize_dataset_batched,
        remove_columns=dataset_processed["train"].column_names,
        batched=True,
        batch_size=BATCH_SIZE,
    )
)

Loading cached processed dataset at /Users/luka/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729/cache-9c485eb4fb0ff28b.arrow


Map:   0%|          | 0/5093 [00:00<?, ? examples/s]

Loading cached processed dataset at /Users/luka/.cache/huggingface/datasets/multi_x_science_sum/default/1.1.0/2876ec0401f8f5c5acf7f4857dbc8d6229a390ab428321ab848f03f14b7f9729/cache-e4d2ef224e30a11e.arrow


In [18]:
dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'global_attention_mask'],
        num_rows: 30369
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'global_attention_mask'],
        num_rows: 5093
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'global_attention_mask'],
        num_rows: 5066
    })
})

In [19]:
# Set up the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./",
    # predict_with_generate=True,
    evaluation_strategy="steps",
    # eval_steps=BATCH_SIZE//2,
    eval_steps=10,
    # save_steps=BATCH_SIZE//2,
    save_steps=10,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    # warmup_steps=BATCH_SIZE,
    # warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    # logging_steps=BATCH_SIZE//4,
    logging_steps=10,
    # use_mps_device=True,
)

In [20]:
# Create a Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized["train"],
    eval_dataset=dataset_tokenized["validation"],
)

In [21]:
%%time

train_result = trainer.train()

/Users/luka/miniconda/envs/nlp_project/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 30369
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 950
  Number of trainable parameters = 152408832


Step,Training Loss,Validation Loss
10,3.970200,3.686373
20,3.715300,3.594553
30,3.653600,3.545779
40,3.636500,3.511709
50,3.582500,3.486015
60,3.559800,3.462022
70,3.536400,3.451576
80,3.528700,3.434519
90,3.523900,3.422537
100,3.517600,3.408048


***** Running Evaluation *****
  Num examples = 5066
  Batch size = 64
Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10/config.json
Configuration saved in ./checkpoint-10/generation_config.json
Model weights saved in ./checkpoint-10/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5066
  Batch size = 64
Saving model checkpoint to ./checkpoint-20
Configuration saved in ./checkpoint-20/config.json
Configuration saved in ./checkpoint-20/generation_config.json
Model weights saved in ./checkpoint-20/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5066
  Batch size = 64
Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoint-30/config.json
Configuration saved in ./checkpoint-30/generation_config.json
Model weights saved in ./checkpoint-30/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 5066
  Batch size = 64
Saving model checkpoint to ./checkpoint-40
Configuration saved in ./checkpoin

CPU times: user 5d 1h 56min 25s, sys: 2d 17h 3min 49s, total: 7d 19h 14s
Wall time: 1d 21h 29min 14s


In [22]:
trainer.log_metrics("train", train_result.metrics)

***** train metrics *****
  epoch                    =                2.0
  total_flos               =         19092699GF
  train_loss               =              3.341
  train_runtime            = 1 day, 21:29:14.07
  train_samples_per_second =              0.371
  train_steps_per_second   =              0.006


In [76]:
%%time

train_result = trainer.train()

***** Running training *****
  Num examples = 16
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 4
  Number of trainable parameters = 152408832


Step,Training Loss,Validation Loss
2,2.253900,4.312594
4,1.969000,4.312594


***** Running Evaluation *****
  Num examples = 16
  Batch size = 8
Saving model checkpoint to ./checkpoint-2
Configuration saved in ./checkpoint-2/config.json
Configuration saved in ./checkpoint-2/generation_config.json
Model weights saved in ./checkpoint-2/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 16
  Batch size = 8
Saving model checkpoint to ./checkpoint-4
Configuration saved in ./checkpoint-4/config.json
Configuration saved in ./checkpoint-4/generation_config.json
Model weights saved in ./checkpoint-4/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




CPU times: user 9min 10s, sys: 7min 41s, total: 16min 52s
Wall time: 3min 58s


In [77]:
trainer.log_metrics("train", train_result.metrics)

***** train metrics *****
  epoch                    =        2.0
  total_flos               =    80472GF
  train_loss               =     2.1585
  train_runtime            = 0:03:58.61
  train_samples_per_second =      0.134
  train_steps_per_second   =      0.017


In [ ]:
led_output_model1 = []
for i in tqdm(range(0, len(dataset_tokenized["test"]["input_ids"]), BATCH_SIZE)):
    
    input_ids = dataset_tokenized["test"]["input_ids"][i:i+BATCH_SIZE]
    attention_mask = dataset_tokenized["test"]["attention_mask"][i:i+BATCH_SIZE]
    global_attention_mask = dataset_tokenized["test"]["global_attention_mask"][i:i+BATCH_SIZE]

    led_output_model1.append(
        model.generate(
            input_ids=torch.as_tensor(input_ids),
            attention_mask=torch.as_tensor(attention_mask),
            global_attention_mask=torch.as_tensor(global_attention_mask),
            no_repeat_ngram_size=3,
            max_length=128,
            num_beams=4,
        )
    )
        
led_output_model1 = torch.cat(led_output_model1)

In [39]:
test_pred_model1 = tokenizer.batch_decode(
    led_output_model1,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

In [40]:
test_pred_model1 = [each.strip() for each in test_pred_model1]

In [41]:
test_pred_model1[:2]

["The long-term goal of our field is the creation and understanding of intelligence. Productive research in AI, both practical and theoretical, benefits from a notion of intelligence that is precise enough to allow the cumulative development of robust systems and general results. This paper outlines a gradual evolution in our formal conception of intelligence, that brings it closer to our informal conception and simultaneously reduces the gap between theory and practice. The article presents experimental results illustrating the agents' dynamic behavior. I. Introduction, 488. — II. The model with automobiles as an example, 489. — III. Examples and applications, 492. — IV.",
 '“Interaction in virtual reality (VR) environments is essential to ensure a pleasant and immersive experience. In this work, we propose a visually realistic, flexible and robust grasping system that enables real-time interactions in virtual environments. Resulting grasps are visually realistic because hand is autom

In [43]:
scores = rouge.compute(
    predictions=test_pred_model1,
    references=dataset_processed["test"]["related_work"],
    use_stemmer=True,
)
scores

{'rouge1': AggregateScore(low=Score(precision=0.2933487638578587, recall=0.29200709832037636, fmeasure=0.2815715983753184), mid=Score(precision=0.31049575936025475, recall=0.30798508735871666, fmeasure=0.29432358485593413), high=Score(precision=0.3280951076358564, recall=0.3250701177873674, fmeasure=0.30592633047634554)),
 'rouge2': AggregateScore(low=Score(precision=0.04350232238478724, recall=0.04349789242499366, fmeasure=0.041407873852706695), mid=Score(precision=0.05183538594707471, recall=0.052840695590428914, fmeasure=0.04971452590462587), high=Score(precision=0.06038594419192698, recall=0.06344514610297083, fmeasure=0.058800723232261924)),
 'rougeL': AggregateScore(low=Score(precision=0.15518199067772578, recall=0.15768831899322766, fmeasure=0.15026841424341691), mid=Score(precision=0.16485430398692857, recall=0.16904587700574214, fmeasure=0.15823517427974215), high=Score(precision=0.1745321488156862, recall=0.18136510192458355, fmeasure=0.16781062293071153)),
 'rougeLsum': Aggr

In [ ]:
# Create a Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

In [ ]:
import torch
from transformers import LongformerTokenizer, EncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments

# Load pre-trained Longformer tokenizer
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

# Load the pre-trained Longformer encoder and decoder
model = EncoderDecoderModel.from_encoder_decoder_pretrained('allenai/longformer-base-4096', 'allenai/longformer-base-4096')

# Define your training and validation data
train_data = ...
val_data = ...

# Set up the training arguments
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=1000,
    num_train_epochs=3,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
)

# Create a Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
)

# Fine-tune the model
trainer.train()


# Sandbox

In [ ]:
%%time

led_output_model1 = model.generate(
    **dataset_tokenized["test"],
    # input_ids=dataset_tokenized["test"]["input_ids"][:n],
    # attention_mask=dataset_tokenized["test"]["attention_mask"][:n],
    # global_attention_mask=dataset_tokenized["test"]["global_attention_mask"][:n],
    no_repeat_ngram_size=3,
    max_length=128,
    num_beams=4,
)

In [41]:
led_output_model1 = []
for i in tqdm(range(0, len(dataset_tokenized["test"]["input_ids"]), BATCH_SIZE)):
    
    input_ids = dataset_tokenized["test"]["input_ids"][i:i+BATCH_SIZE]
    attention_mask = dataset_tokenized["test"]["attention_mask"][i:i+BATCH_SIZE]
    global_attention_mask = dataset_tokenized["test"]["global_attention_mask"][i:i+BATCH_SIZE]

    led_output_model1.append(
        model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            global_attention_mask=global_attention_mask,
            no_repeat_ngram_size=3,
            max_length=128,
            num_beams=4,
        )
    )
        
led_output_model1 = torch.cat(led_output_model1)

100%|████████████████████████████████████████████████████████████████████████████████████| 80/80 [3:11:58<00:00, 143.98s/it]


In [42]:
test_pred_model1 = tokenizer.batch_decode(
    led_output_model1,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=True,
)

In [43]:
test_pred_model1 = [each.strip() for each in test_pred_model1]

In [44]:
test_pred_model1[:2]

["The long-term goal of our field is the creation and understanding of intelligence. Productive research in AI, both practical and theoretical, benefits from a notion of intelligence that is precise enough to allow the cumulative development of robust systems and general results. This paper outlines a gradual evolution in our formal conception of intelligence, that brings it closer to our informal conception and simultaneously reduces the gap between theory and practice. The article presents experimental results illustrating the agents' dynamic behavior. I. Introduction, 488. — II. The model with automobiles as an example, 489. — III. Examples and applications, 492. — IV.",
 '“Interaction in virtual reality (VR) environments (e.g. grasping and manipulating virtual objects) is essential to ensure a pleasant and immersive experience. In this work, we propose a visually realistic, flexible and robust grasping system that enables real-time interactions in virtual environments. Resulting gr

In [45]:
scores = rouge.compute(
    predictions=test_pred_model1,
    references=dataset_processed["test"]["related_work"],
    use_stemmer=True,
)
scores

{'rouge1': AggregateScore(low=Score(precision=0.2982484009370646, recall=0.3057877687297647, fmeasure=0.2877549383290699), mid=Score(precision=0.3011000320975565, recall=0.308186529440017, fmeasure=0.28974742885739196), high=Score(precision=0.30423223116720244, recall=0.31058012470155627, fmeasure=0.2917896849338474)),
 'rouge2': AggregateScore(low=Score(precision=0.046860562392848866, recall=0.04742509910853629, fmeasure=0.04476525721466253), mid=Score(precision=0.04805986457733996, recall=0.048611032584090635, fmeasure=0.0458330562332894), high=Score(precision=0.04916571328652274, recall=0.0498130873004002, fmeasure=0.046871122190935706)),
 'rougeL': AggregateScore(low=Score(precision=0.15540080811555254, recall=0.1636225890270709, fmeasure=0.15142010390078578), mid=Score(precision=0.15687893432752667, recall=0.16523845626801453, fmeasure=0.1524776573706927), high=Score(precision=0.15834376408384634, recall=0.16680563034698537, fmeasure=0.15364196168214198)),
 'rougeLsum': AggregateS